In [5]:
import pandas as pd
import numpy as np
from pickle import dump

In [4]:
text = pd.read_excel(r"C:\Users\Thomas.Zoellinger\Downloads\MYDF2.xlsx", sep=";",header="infer", encoding ="UTF-8", usecols = ["Hauptartikel"])


In [8]:
text = text["Hauptartikel"].str.cat(sep='\n')

In [9]:
chars = sorted(list(set(text)))
mapping = dict((c, i) for i, c in enumerate(chars))
dump(mapping, open('mapping.pkl', 'wb'))

vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

encoded_text = [mapping[char] for char in text]
encode_size = len(encoded_text)
print('Code Size: %d' % encode_size)

Vocabulary Size: 129
Code Size: 22901050


In [11]:
seqlen = 10
batchsize = 512
batchnum = int((encode_size - seqlen) / batchsize)

from keras.utils import to_categorical

def myGenerator():
    while 1:
        for i in range(batchnum): 
            X_batch = []
            y_batch = []
            for j in range(batchsize):
                X_batch.append(encoded_text[i*batchsize+j:i*batchsize+j+seqlen])
                y_batch.append(encoded_text[i*batchsize+j+seqlen:i*batchsize+j+seqlen+1])
                
            X_batch = np.array([to_categorical(x, num_classes=vocab_size) for x in X_batch])
            y_batch = np.array(to_categorical(y_batch, num_classes=vocab_size))

            yield (X_batch, y_batch)

Using TensorFlow backend.


In [12]:
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, SimpleRNN
from keras.models import Model

model = Sequential()
model.add(LSTM(300, return_sequences=True, input_shape=(seqlen, vocab_size)))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(75))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

W0805 19:17:58.887475 19836 deprecation_wrapper.py:119] From c:\users\thomas.zoellinger\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 19:17:58.902014 19836 deprecation_wrapper.py:119] From c:\users\thomas.zoellinger\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 19:17:58.906459 19836 deprecation_wrapper.py:119] From c:\users\thomas.zoellinger\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 300)           516000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 150)           270600    
_________________________________________________________________
lstm_3 (LSTM)                (None, 75)                67800     
_________________________________________________________________
dense_1 (Dense)              (None, 129)               9804      
Total params: 864,204
Trainable params: 864,204
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
my_generator = myGenerator()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit_generator(my_generator, steps_per_epoch = batchnum, epochs = 1, verbose=1)
model.save('model_3lay.h5')

W0805 19:18:31.217976 19836 deprecation_wrapper.py:119] From c:\users\thomas.zoellinger\appdata\local\programs\python\python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0805 19:18:31.249572 19836 deprecation_wrapper.py:119] From c:\users\thomas.zoellinger\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0805 19:18:31.363640 19836 deprecation.py:323] From c:\users\thomas.zoellinger\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0805 19:18:32.941306 19836 deprecation_wrapper.py:119] From c:\users\thomas.zoell

Epoch 1/1
  315/44728 [..............................] - ETA: 4:41:47 - loss: 3.3173 - acc: 0.1406

KeyboardInterrupt: 